<a href="https://colab.research.google.com/github/josedavidmarin/C-S/blob/main/Nota2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evaluación: Análisis Sintáctico en Compiladores
**Fecha:** 2025-05-09

---

## Tema 1: Gramáticas Libres del Contexto

**1.1.** Defina qué es una gramática libre del contexto (GLC) y explique cómo se diferencia de una gramática regular.

Una Gramática Libre del Contexto (GLC) es una gramática formal en la que todas las producciones son de la forma:
```
𝐴→𝛼
```
donde A es un no terminal y α es una cadena de terminales y/o no terminales. La producción no depende del contexto en el que se encuentre A.

Diferencias con una gramática regular:

En una gramática regular, las producciones tienen una forma más restringida: 𝐴→𝑎𝐵 o 𝐴→𝑎, donde a es un terminal y B un no terminal.

Las gramáticas regulares solo pueden generar lenguajes regulares, mientras que las GLC generan lenguajes más complejos, como los lenguajes de paréntesis balanceados.


**1.2.** Dada la siguiente gramática:
```
S → aSb | ε
```
a) ¿Qué lenguaje genera?

Genera cadenas con la misma cantidad de a y b, en el formato:
```
𝐿={𝑎^𝑛 𝑏^𝑛∣𝑛≥0}
```

Ejemplos: ε, ab, aabb, aaabbb...

b) ¿Es ambigua? Justifique.

Cada cadena tiene una única derivación. Por ejemplo, para "aabb", solo se puede derivar así:
```
S⇒aSb⇒aaSbb⇒aaεbb=aabb
```
c) Escriba una derivación izquierda y una derivación derecha para la cadena `aabb`.

Izquierda:
```
S⇒aSb⇒aaSbb⇒aaεbb=aabb
```

Derecha:
```
S⇒aSb⇒a(aSb)b⇒a(aεb)b=aabb
```
**1.3.** Diseñe una GLC que genere el lenguaje de todas las cadenas de paréntesis correctamente balanceadas, como `(()())`.

Una posible GLC:
```
S→(S)S∣ε
```
Ejemplos generados: ε, (), ()(), (()), (()()), etc


## Tema 2: Análisis Descendente - LL(k), FIRST y FOLLOW

**2.1.** Explique en qué consiste un analizador LL(k). ¿Por qué el análisis LL(1) es tan popular en la práctica?

Un analizador LL(k) es un analizador descendente (top-down) que construye el árbol de derivación desde la raíz (símbolo inicial) hacia las hojas. Lee la entrada de izquierda a derecha (L), produce una derivación izquierda (L) y usa k-símbolos de lookahead para decidir qué producción aplicar.

Usa una pila para mantener el estado del análisis.
Se basa en una tabla LL(k) construida a partir de los conjuntos FIRST y FOLLOW para seleccionar producciones.

Popularidad de LL(1):

Simplicidad: Requiere solo un símbolo de lookahead, lo que simplifica la construcción de la tabla y reduce la complejidad computacional.
Eficiencia: Los analizadores LL(1) son rápidos y tienen un tamaño de tabla manejable, adecuados para lenguajes de programación simples.
Implementación directa: Facilita la escritura manual de analizadores recursivos descendentes.
Limitaciones: No todas las gramáticas son LL(1) (p.ej., gramáticas ambiguas o con recursión izquierda), pero muchas gramáticas de lenguajes prácticos pueden transformarse para serlo.


**2.2.** Dada la gramática:
```
E → T E'
E' → + T E' | ε
T → F T'
T' → * F T' | ε
F → ( E ) | id
```
a) Calcule los conjuntos **FIRST** y **FOLLOW** para cada no terminal.

* FIRST(E) = FIRST(T) = FIRST(F) = {(, id}

  FIRST(E') = {+, ε}

  FIRST(T') = {*, ε}



* FOLLOW(E) = { ')', $ }

  FOLLOW(E') = FOLLOW(E) = { ')', $ }

  FOLLOW(T) = FIRST(E') ∪ FOLLOW(E') = { '+', ')', $ }

  FOLLOW(T') = FOLLOW(T) = { '+', ')', $ }

  FOLLOW(F) = FIRST(T') ∪ FOLLOW(T') = { '*', '+', ')', $ }


b) Construya la tabla LL(1).

| No Terminal | (        | id       | +           | \*           | )      | \$     |
| ----------- | -------- | -------- | ----------- | ------------ | ------ | ------ |
| E           | E → T E' | E → T E' |             |              |        |        |
| E'          |          |          | E' → + T E' |              | E' → ε | E' → ε |
| T           | T → F T' | T → F T' |             |              |        |        |
| T'          |          |          | T' → ε      | T' → \* F T' | T' → ε | T' → ε |
| F           | F → (E)  | F → id   |             |              |        |        |



c) ¿La gramática es LL(1)? Justifique.

La gramática es LL(1).

Una gramática es LL(1) si su tabla LL(1) no tiene conflictos (ninguna celda con múltiples producciones).
La tabla construida no tiene celdas con más de una entrada, lo que indica que para cada combinación de no terminal y símbolo de entrada, hay a lo sumo una producción aplicable.
Además, la gramática no tiene recursión izquierda y los conjuntos FIRST de las producciones de cada no terminal son disjuntos (o usan FOLLOW correctamente para ε-producciones).
Por lo tanto, la gramática satisface las condiciones para ser LL(1).


## Tema 3: Análisis Ascendente - LR(0), SLR, LR(1), LALR(1)

**3.1.** Explique brevemente la diferencia entre los siguientes analizadores:

a) LR(0)

Descripción: Usa un autómata de estados basado en items LR(0) (producciones con un punto, sin lookahead). Decide shift o reduce basándose solo en el estado actual.

Ventajas: Simple, fácil de construir, requiere poca memoria.

Desventajas: Muy restrictivo, genera muchos conflictos shift/reduce y reduce/reduce porque no usa información de contexto (lookahead). Solo sirve para gramáticas muy simples.

b) SLR

Descripción: Extiende LR(0) usando los conjuntos FOLLOW para decidir reducciones. Un item 𝐴→𝛼 en un estado permite reducir si el siguiente símbolo está en FOLLOW(A).

Ventajas: Más poderoso que LR(0), resuelve algunos conflictos shift/reduce usando FOLLOW. Más fácil de implementar que LR(1).

Desventajas: Puede generar conflictos si FOLLOW incluye símbolos que no son válidos en el contexto del estado, ya que no considera lookahead específico.

c) LR(1)

Descripción: Usa items LR(1) de la forma [𝐴→𝛼*β, a] es un símbolo de lookahead. Las reducciones se hacen solo si el siguiente símbolo coincide con el lookahead del item.

Ventajas: Muy poderoso, puede manejar gramáticas más complejas sin conflictos. Resuelve muchos conflictos de SLR.

Desventajas: Las tablas son mucho más grandes (más estados debido a los lookaheads), lo que aumenta la complejidad y el uso de memoria.

d) LALR(1)

Descripción: Es un compromiso entre SLR y LR(1). Usa el autómata LR(1), pero combina estados con los mismos items LR(0), uniendo sus lookaheads.

Ventajas: Más eficiente que LR(1) (tablas más pequeñas), más poderoso que SLR (usa lookaheads más precisos). Ampliamente usado en herramientas como Yacc/Bison.

Desventajas: Puede introducir conflictos reduce/reduce que no estaban en LR(1) debido a la combinación de lookaheads, aunque esto es raro en la práctica.

Indique qué ventajas y desventajas tiene cada uno.

Resumen:

LR(0) es el más simple pero menos poderoso.
SLR mejora LR(0) con FOLLOW, pero es menos preciso.
LR(1) es el más poderoso, pero las tablas son grandes.
LALR(1) balancea potencia y eficiencia, siendo el más usado en la práctic


**3.2.** Para la siguiente gramática:
```
S → L = R | R
L → * R | id
R → L
```
a) Construya el conjunto de **items LR(0)**.


Estado 0 (inicial):

S' → •S

S → •L = R

S → •R

L → •* R

L → •id

R → •L

Estado 1 (desde 0 con S):

S' → S•

Estado 2 (desde 0 con L):

S → L•= R

R → L•

Estado 3 (desde 0 con R):

S → R•

*Estado 4 (desde 0 con ):

L → *•R

R → •L

L → •* R

L → •id

Estado 5 (desde 0 con id):

L → id•

Estado 6 (desde 2 con =):

S → L=•R

R → •L

L → •* R

L → •id

Estado 7 (desde 4 con R):

L → *R•

Estado 8 (desde 4 o 6 con L):

R → L•

Estado 9 (desde 6 con R):

S → L=R•


b) Construya el autómata LR(0) de estados.

Estado 0:

  S'→•S, S→•L=R, S→•R, L→•*R, L→•id, R→•L

  *→4, id→5, S→1, L→2, R→3
  
  

Estado 1 (final):

  S'→S•

  ACCEPT


Estado 2:

  S→L•=R, R→L•

  =→6, $→reduce R→L (conflicto)


Estado 3:

  S→R•

  $→reduce S→R

Estado 4:

  L→*•R, R→•L, L→•*R, L→•id

  *→4, id→5, R→7, L→8

Estado 5:

  L→id•

  =/$→reduce L→id

Estado 6:

  S→L=•R, R→•L, L→•*R, L→•id

  *→4, id→5, R→9, L→8

Estado 7:

  L→*R•

  =/$→reduce L→*R

Estado 8:

  R→L•

  =/$→reduce R→L

Estado 9:

  S→L=R•

  $→reduce S→L=R

c) Indique si hay conflictos shift/reduce o reduce/reduce en la tabla LR(0).

Conflicto en Estado 2:

S → L • = R  
R → L •


Si el siguiente símbolo es =, el parser quiere hacer shift a =.

Pero también podría reducir con R → L.


**3.3.** Explique qué problema resuelve el enfoque LALR(1) y cómo se construyen sus tablas a partir de las de LR(1).

Problema que resuelve:
LALR(1) (Look-Ahead LR) resuelve el problema del tamaño excesivo de las tablas LR(1) manteniendo casi la misma potencia de análisis. Combina estados que tienen el mismo núcleo LR(0) pero diferentes conjuntos de lookahead, reduciendo significativamente el tamaño de las tablas (típicamente a un tamaño similar a SLR).

Construcción de tablas LALR(1) a partir de LR(1):
Construir los ítems LR(1) completos:

Cada ítem lleva un conjunto de símbolos de lookahead

Identificar estados con mismo núcleo LR(0):

Dos estados tienen el mismo núcleo si sus ítems difieren solo en los lookaheads

Fusionar estados compatibles:

Unir todos los ítems con el mismo núcleo

Combinar sus conjuntos de lookahead

Construir la tabla con los estados fusionados:

Las transiciones se ajustan para reflejar los estados fusionados

Las acciones se combinan, verificando que no haya conflictos

Ejemplo de fusión:
Si en LR(1) tenemos:

Estado A: [X→α•β, {a}]

Estado B: [X→α•β, {b}]

En LALR(1) se fusionan en:

Estado AB: [X→α•β, {a,b}]

Ventajas:

Tablas mucho más pequeñas que LR(1) (similar tamaño a SLR)

Mayor poder de análisis que SLR

Detecta errores sintácticos tan pronto como LR(1)

Desventajas:

Puede introducir conflictos que no existían en LR(1)

Algunas gramáticas son LR(1) pero no LALR(1)
